In [109]:
import numpy as np
from numpy.lib.stride_tricks import as_strided as ast
from pathlib import Path
import cv2
import math
import itertools
import random

In [135]:
def pot_grid(img_shape, num_pots):
    imght, imgwid = img_shape[:2]
    # adj_pots = random.randint(0, pots)

    if num_pots >= 10:
        num = random.choice([2,3])
        wid = list(np.linspace(0,imgwid,4 if num==3 else 5, dtype=int))
        ht = list(np.linspace(0, imght, num, dtype=int))

    if num_pots == 11:
        pops =+ 1 if num == 3 else 0

    if num_pots == 10:
        pops =+ 2 if num == 3 else 0

    if num_pots == 9:
        num = random.choice([2,3])
        wid = list(np.linspace(0,imgwid,3 if num==3 else 5, dtype=int))
        ht = list(np.linspace(0, imght, num, dtype=int))
        pops =+ 1 if num==2 else 0


    coords = list(itertools.product(wid, ht))
        
    for po in range(pops):
        coords.pop(random.randrange(len(coords)))
    
    return coords

2
[0, 2390, 4780, 7170, 9560]
[0, 6368]

[(0, 0), (0, 6368), (2390, 0), (2390, 6368), (4780, 0), (4780, 6368), (7170, 0), (7170, 6368), (9560, 0), (9560, 6368)]
10


In [122]:
num

2